
## rules seem to work but have redundancies. OK for now but maybe better to simplify and select only operators that are generators (need to work them out).

## next: look at logic for tensors and rules. look at ANFform in sympy

## ToDo

SG: 161: two coordinates are related by Laue sym.

get two trans rules. are they equivalent? (yes, probably: Rule A - Rule B equiv to Laue Rule B.

if so, is there always only one trans rule (after Laue sym)? maybe... assume yes

then sort out centring rules (allow for zero, one, more than one?) ... assume zero or one for now

the wrap up rules in a dict and create function to find if ref is allowed ######### next

isAllowedByTensorCalk(hkl, sg, wyck)

isAllowedByCondRules(hkl, condRules)

then we have confidence to look for tensor rules...


---------


Remove redundant/equivalent rules

    In simplify_rule replace trivially even expressions (2*l, 6*l etc) with 0 - done (??)
    
    remove duplicate rules to form short set - done (?)
    
    next (Sunday): check output against Int Tables - done: correct with 4 x laue redundancies and 1 x numerical redundancy (l/2 and l)
    
    remove laue redundancies - done first attempt: check Int Tables and re-order (OK; don't re-order - tidying up for later)

    check with some other groups
    
    generate reflections from rules
    
    find match to rules for different classes

use closed world assumption for tensor rules





## Questions 
Are translational rules identical to rules with no conditions?

## Notes
Nothing relating to Wyckoff letter yet

In [1]:
#imports & setup
import cctbx.sgtbx as sg
import sympy as sp

x, y, z = sp.symbols(['x', 'y', 'z'])
h, k, l = sp.symbols(['h', 'k', 'l'])

In [2]:
#config
sgNum = 142 #ok
#sgNum = 37  #ok
#sgNum = 1 # error - no trans rule (primative) 
#sgNum = 230 #ok but some redundancies
#sgNum = 161 # ok
#sgNum = 194 # error - no trans rule (primative) 

In [3]:
sgi = sg.space_group_info(sgNum)
spaceGroup = sgi.group()
print(sgi.symbol_and_number())
symxyz = [str(s.as_xyz()) for s in spaceGroup.all_ops()]
print(symxyz)

laueSymList = [op.as_xyz().replace('x', 'h').replace('y', 'k').replace('z', 'l') for op in spaceGroup.build_derived_laue_group().all_ops()]
print(laueSymList)

I 41/a c d :2 (No. 142)
['x,y,z', '-y+1/4,x+3/4,z+1/4', 'y+1/4,-x+1/4,z+3/4', 'x,-y,-z+1/2', '-x+1/2,y,-z', '-x,-y+1/2,z', 'y+1/4,x+3/4,-z+3/4', '-y+1/4,-x+1/4,-z+1/4', '-x,-y,-z', 'y-1/4,-x-3/4,-z-1/4', '-y-1/4,x-1/4,-z-3/4', '-x,y,z-1/2', 'x-1/2,-y,z', 'x,y-1/2,-z', '-y-1/4,-x-3/4,z-3/4', 'y-1/4,x-1/4,z-1/4', 'x+1/2,y+1/2,z+1/2', '-y+3/4,x+5/4,z+3/4', 'y+3/4,-x+3/4,z+5/4', 'x+1/2,-y+1/2,-z+1', '-x+1,y+1/2,-z+1/2', '-x+1/2,-y+1,z+1/2', 'y+3/4,x+5/4,-z+5/4', '-y+3/4,-x+3/4,-z+3/4', '-x+1/2,-y+1/2,-z+1/2', 'y+1/4,-x-1/4,-z+1/4', '-y+1/4,x+1/4,-z-1/4', '-x+1/2,y+1/2,z', 'x,-y+1/2,z+1/2', 'x+1/2,y,-z+1/2', '-y+1/4,-x-1/4,z-1/4', 'y+1/4,x+1/4,z+1/4']
['h,k,l', '-k,h,l', '-h,-k,l', 'k,-h,l', 'h,-k,-l', 'k,h,-l', '-h,k,-l', '-k,-h,-l', '-h,-k,-l', 'k,-h,-l', 'h,k,-l', '-k,h,-l', '-h,k,l', '-k,-h,l', 'h,-k,l', 'k,h,l']


In [4]:
def eliminate_xyz(ff):

    u = {'h': sp.symbols('u_h'), 'k': sp.symbols('u_k'), 'l': sp.symbols('u_l')}

    def find_xyz(expression):
        if expression.coeff(x) != 0:
            return x
        if expression.coeff(y) != 0:
            return y
        if expression.coeff(z) != 0:
            return z

    for hkl in ['h', 'k', 'l']:
        expr =  ff.coeff(hkl)
        xyz_var = find_xyz(expr)

        if xyz_var != None:
            ff = ff.subs(xyz_var, sp.solve(expr - u[hkl], xyz_var)[0])

        conds = []
        for hkl in ['h', 'k', 'l']:
            coef = ff.expand().coeff(u[hkl])
            if coef != 0:
                conds += [sp.Eq(ff.expand().coeff(u[hkl]), 0)]

        #rule: this expression is odd
        rule = ff.subs({u['h']:0, u['k']:0, u['l']:0})
            
    return (ff, set(conds), rule) # use set to remove duplicates
    

def condRulesFromSymOps(symxyz):

    cond_rule_list = []
    
    for sym in symxyz:
        xp, yp, zp = sp.simplify(sym)
        ff = 2 * h * (xp - x) + 2 * k * (yp - y) + 2 * l * (zp - z)
        ff, conds, rule = eliminate_xyz(ff)
        #display('sym:', sym, 'ff:', ff, 'conds:', conds, 'rule:', rule)
        #print('\n')
        if rule != 0:
            if [[conds, rule]] not in cond_rule_list:
                cond_rule_list += [[conds, rule]]
                #print('??? cond, rule, sym:', [conds, rule], sym) # delete print
    return cond_rule_list

def condRulesFromSpacegroup(sgNum):
    sgi = sg.space_group_info(sgNum)
    spaceGroup = sgi.group()
    #print(sgi.symbol_and_number())
    symxyz = [str(s.as_xyz()) for s in spaceGroup.all_ops()]
    return symxyz

def separateTransRule(cond_rule_list):
    # return None for transRule if no centring rules
    transRule, other_rules = None, []
    for _cond_rule in cond_rule_list:
        if _cond_rule[0] == set():       # pick out translational rule (last one - might be bettwe with first one?)
            transRule = _cond_rule[1]
        else:
            other_rules += [_cond_rule]  # if not trans rule then add to other rule list
    return (transRule, other_rules)

# def simplify_rule(rule):
#     # simplify rule for even expression e.g 3l/2 even same as l/2 even ### ??? check
#     h, k, l = sp.symbols(['h', 'k', 'l'])
    
#     new_rule = sp.Integer(0)
#     for hkl in [h, k, l]:
#         c = rule.coeff(hkl)
#         (num, denom) = sp.simplify(sp.fraction(c))

#         if denom != 1: # its a fraction
#             num1 = (num + 2 * denom) % (2 * denom) - (2 * denom)
#         else:
#             num1 = num % 2
#         cnew = sp.Rational(num1,denom)
#         new_rule += hkl * cnew
        
#     #print('Old rule: ', rule, 'New rule: ', new_rule) ########### remove after testing
      
#     return sp.simplify(new_rule)

def simplify_rule(rule):
    # updated version of previous simplify_rule (which had a bug). This one works for single miller index
    # not applied if more than one index - might want to create a better algorithm.
    h, k, l = sp.symbols(['h', 'k', 'l'])
    
    new_rule = rule.subs(((h, 0), (k, 0), (l, 0))) # get constant part

    for hkl in [h, k, l]:
        c = rule.coeff(hkl)
        if c != 0:  #skip if no coefficient for this miller index (h, k or l)

            (num, denom) = sp.simplify(sp.fraction(2/c))

            cnew = sp.Abs(sp.Rational(2, num)) # positive value is equivalent
            if sp.Abs(cnew) > 1: # rule is trivial - set to zero
                cnew = 0
            
            new_rule += hkl * cnew
        
    #print('Old rule: ', rule, 'New rule: ', new_rule) ########### remove after testing
    
    # don't simplify if more than one non-zero coefficient (haven't figured out how to yet!)
        
    #print(rule.coeff(h), rule.coeff(k), rule.coeff(l))
    #print(int(rule.coeff(h) != 0), int(rule.coeff(k) != 0), int(rule.coeff(l) != 0))
    
    if int(rule.coeff(h) != 0) + int(rule.coeff(k) != 0) + int(rule.coeff(l) != 0) > 1:
        return sp.simplify(rule)
    else:
        return sp.simplify(new_rule)




def ss(n, d):
    if d == 1:
        n1 = n % 2
    else:
        #n1 = n % (2 * d) # always +ve
        n1 = (n + 2 * d) % (2 * d) - (2 * d) # between -1 and +1 (but -1 for 1,1)
    return n1



def simplify_rules(cond_rule_list):
    for cond_rule in cond_rule_list:
        cond_rule[1] = simplify_rule(cond_rule[1])
    return cond_rule_list

def remove_trivial_rules(cond_rule_list):
    
    new_cond_rule_list = []
    
    for cond_rule in cond_rule_list:
        #print('cond_rule:', cond_rule)
        for hkl in ['h', 'k', 'l']:
            if cond_rule[1].coeff(hkl) % 2 == 0:
                #print('before:', cond_rule[1])
                cond_rule[1] = cond_rule[1].subs({hkl: 0})
                #print('after:', cond_rule[1])
        if cond_rule[1] != 0:
            new_cond_rule_list += [cond_rule]

    return new_cond_rule_list

def remove_duplicate_rules(cond_rule_list):
    new_cond_rule_list = []
    
    for cond_rule in cond_rule_list:
        _new = True
        for new_cond_rule in new_cond_rule_list:
            if cond_rule == new_cond_rule:
                _new = False
                break
        if _new:
            new_cond_rule_list += [cond_rule]      
            
    return new_cond_rule_list

def apply_laueSym_to_cond_rule(cond_rule, laueSym):
    
    condset, rule = cond_rule
    subs = {'h': laueSym[0], 'k': laueSym[1], 'l': laueSym[2]}
    
    new_condset = set()
    
    for cond in condset:
        new_condset.add(sp.simplify(cond.subs(subs, simultaneous=True)))
    
    rule = sp.simplify(simplify_rule(rule.subs(subs, simultaneous=True)))
    return [new_condset, rule]
    
    

def remove_laue_duplicates(cond_rule_list, laueSymList):
    # might be better to re-order list first?
    new_cond_rule_list = []
    
    for cond_rule in cond_rule_list:
        _new = True
        for laue in laueSymList:
            
            new_cond_rule = apply_laueSym_to_cond_rule(cond_rule, laue)
            
            for _cr in new_cond_rule_list:
                if _cr == new_cond_rule:
                    _new = False
                    break
        if _new:
            new_cond_rule_list += [cond_rule]

        
    return new_cond_rule_list   


# def isAllowedByCondRules(hklList, rulesDict):
#     laueSymList, transRule, nonTransCondRules =   rulesDict['laueSymList'], rulesDict['transRule'], rulesDict['nonTransCondRules'] 
    
#     hklAllowed = []

#     #start with transrule ... seems to work but slow ... maybe put laue in outser loop?
    
#     for hkl in hklList:
#         #print(hkl)
#         allowed = True # assume true unless a rule is violated
#         # centring rule
#         for laue in laueSymList:

#             dummy, transRule =  apply_laueSym_to_cond_rule([{}, rulesDict['transRule']], laue)
                       
#             if sp.N(transRule.subs({h:hkl[0], k:hkl[1], l:hkl[2]})) % 2 != 0: # expression not even; rule broken
#                 allowed = False
#                 #print('False because trans rule failed: ', transRule, 'laue transrule:', transRule.subs({h:hkl[0], k:hkl[1], l:hkl[2]}))
#                 break
                
#             for nonTransCondRule in nonTransCondRules:
#                 conds, rule =  apply_laueSym_to_cond_rule(rulesDict['nonTransCondRules'][i], laue)
#                 #conds, rule =  apply_laueSym_to_cond_rule(nonTransCondRule, laue) # bug - replaced with line above
                                                        
#                 for cond in conds:
#                     condhkl = cond.subs({h:hkl[0], k:hkl[1], l:hkl[2]}) # evaluation for hkl
#                     if not condhkl:   # skip if any condition violated
#                         break                                                        
                                                       
#                 if condhkl: #condition is True
#                     if sp.N(rule.subs({h:hkl[0], k:hkl[1], l:hkl[2]}, simultaneous=True)) % 2 != 0: # expression not even; rule broken
#                         allowed = False
#                         #print('False because non-trans rule failed. laue:', laue, 'original conds:', conds, 'original rule: ', rule )

#                         break          
                                                        
            
#         #print(hkl, allowed)
#         hklAllowed += [allowed]
    
#     return hklAllowed


# def getTruthListfromReflist(hklList, rulesDict):
#     #display('rulesDict:', rulesDict)
#     h, k, l = sp.symbols(['h', 'k', 'l'])
#     truthListLength = 1 + len(rulesDict['nonTransCondRules'])
#     truthDict = dict()
    
#     for hkl in hklList:

#         truthList = [True] * truthListLength
            
#         for laue in laueSymList: # might not be necessary to apply laue to transrule
#             dummy, transRule =  apply_laueSym_to_cond_rule([{}, rulesDict['transRule']], laue)
#             if sp.N(transRule.subs({h:hkl[0], k:hkl[1], l:hkl[2]})) % 2 != 0: # expression not even; rule broken
#                 truthList[0] = False
#                 break

#         for i in range(len(rulesDict['nonTransCondRules'])):
#             for laue in laueSymList:
#                 #conds, rule =  apply_laueSym_to_cond_rule(nonTransCondRules[i], laue)
#                 conds, rule =  apply_laueSym_to_cond_rule(rulesDict['nonTransCondRules'][i], laue)
#                 #display("rulesDict['nonTransCondRules'][i], conds (laue), rule(laue)", rulesDict['nonTransCondRules'][i], conds, rule)
#                 for cond in conds:                       
#                     condhkl = cond.subs({h:hkl[0], k:hkl[1], l:hkl[2]}) # evaluation for hkl
#                     #print(hkl, cond, condhkl)

#                     if not condhkl:   # skip if any condition violated
#                         break        # break out of conds loop as soon as one fails                                                
                                                       
#                 #print(i, conds, cond, rule, condhkl)
#                 if condhkl: #condition is True
#                     if sp.N(rule.subs({h:hkl[0], k:hkl[1], l:hkl[2]}, simultaneous=True)) % 2 != 0: # expression not even; rule broken
#                         truthList[i+1] = False
#                         break   # break out of laue loop as soon as a rule fails
#                 #print(list(map(int, truthList)))
#             #print('R%i' % (i+2), truthList[i+1])
#         truthDict[tuple(hkl)] = truthList
#         #print(truthList)
#     return truthDict

def getTruthListfromReflist(hklList, rulesDict):
    #display('rulesDict:', rulesDict)
    h, k, l = sp.symbols(['h', 'k', 'l'])
    truthListLength = 1 + len(rulesDict['nonTransCondRules'])
    truthDict = dict()
    
    for hkl in hklList:

        truthList = [True] * truthListLength
            
        for laue in laueSymList: # might not be necessary to apply laue to transrule
            dummy, transRule =  apply_laueSym_to_cond_rule([{}, rulesDict['transRule']], laue)
            if sp.N(transRule.subs({h:hkl[0], k:hkl[1], l:hkl[2]})) % 2 != 0: # expression not even; rule broken
                truthList[0] = False
                break

        for i in range(len(rulesDict['nonTransCondRules'])):
            for laue in laueSymList:
                #conds, rule =  apply_laueSym_to_cond_rule(nonTransCondRules[i], laue)
                conds, rule =  apply_laueSym_to_cond_rule(rulesDict['nonTransCondRules'][i], laue)
                #display("rulesDict['nonTransCondRules'][i], conds (laue), rule(laue)", rulesDict['nonTransCondRules'][i], conds, rule)
                for cond in conds:                       
                    condhkl = cond.subs({h:hkl[0], k:hkl[1], l:hkl[2]}) # evaluation for hkl
                    #print(hkl, cond, condhkl)

                    if not condhkl:   # skip if any condition violated
                        break        # break out of conds loop as soon as one fails                                                
                                                       
                #print(i, conds, cond, rule, condhkl)
                if condhkl: #condition is True
                    if sp.N(rule.subs({h:hkl[0], k:hkl[1], l:hkl[2]}, simultaneous=True)) % 2 != 0: # expression not even; rule broken
                        truthList[i+1] = False
                        break   # break out of laue loop as soon as a rule fails
                #print(list(map(int, truthList)))
            #print('R%i' % (i+2), truthList[i+1])
        truthDict[tuple(hkl)] = truthList
        #print(truthList)
    return truthDict


def isAllowedByTensorCalc(hklList, sg_no, wyckoff_letter = None):
    import TensorScatteringClass as ten
    
    t=ten.TensorScatteringClass(spacegroup_number = sg_no, wyckoff_letter = 'a', verbose = False)
    if wyckoff_letter == None:
        wyckoff_letter = t.all_letters[0]
        print(t.all_letters, wyckoff_letter)
      
        hklAllowed = []
        
        for hkl in hklList:
            t.SF_symmetry(t.sitevec, hkl, t.sglist)
            hklAllowed += [t.gen_scalar_allowed == 1]
    
    return hklAllowed
    
def genReflist(hklMax):
    refList = []
    for h in range(-hklMax, hklMax+1):
        for k in range(-hklMax, hklMax+1):
            for l in range(-hklMax, hklMax+1):
                 refList += [[h, k, l]]
    return refList
                    
def laueClassEquivRefs(hkl, laueSymList):       
    h, k, l = hkl
    hklNewList = []
    for laue in laueSymList:

            newhkl = [eval(laue[0]), eval(laue[1]), eval(laue[2])]
            hklNewList += [newhkl]
    return hklNewList
       

#### test this    
def laueSymListFromSpacegroup(sgNum):
    from sympy.parsing.sympy_parser import parse_expr as p
    from sympy.matrices import Matrix
    
    spaceGroup = sg.space_group_info(sgNum).group()
    
    ashklList = [] 
    for op in spaceGroup.build_derived_laue_group().all_ops():
        asxyz = op.as_xyz().split(',')
   
        m = sp.Matrix([
            [p(asxyz[0]).coeff('x'), p(asxyz[0]).coeff('y'), p(asxyz[0]).coeff('z')],
            [p(asxyz[1]).coeff('x'), p(asxyz[1]).coeff('y'), p(asxyz[1]).coeff('z')],
            [p(asxyz[2]).coeff('x'), p(asxyz[2]).coeff('y'), p(asxyz[2]).coeff('z')]])

        #minv = m.inv() # doesn't work - try transpose - looks ok? but transpose without inv may be ok??
        minv = m.inv().T
        
        #v = sp.simplify(minv * sp.Matrix(['x', 'y', 'z']))
        vhkl = sp.simplify(minv * sp.Matrix(['h', 'k', 'l']))
        
        
        ashklList += [[str(vhkl[0]),str(vhkl[1]),str(vhkl[2])]]

    return(ashklList)
        
    
# def getTruthListfromReflist(hklList, rulesDict):
#     h, k, l = sp.symbols(['h', 'k', 'l'])
#     truthListLength = 1 + len(rulesDict['nonTransCondRules'])
#     truthDict = dict()
    
#     for hkl in hklList:

#         truthList = [True] * truthListLength
            
#         for laue in laueSymList: # might not be necessary to apply laue to transrule
#             dummy, transRule =  apply_laueSym_to_cond_rule([{}, rulesDict['transRule']], laue)
#             if sp.N(transRule.subs({h:hkl[0], k:hkl[1], l:hkl[2]})) % 2 != 0: # expression not even; rule broken
#                 truthList[0] = False
#                 break

#         for i in range(len(rulesDict['nonTransCondRules'])):
#             for laue in laueSymList:
#                 conds, rule =  apply_laueSym_to_cond_rule(nonTransCondRules[i], laue)
#                 for cond in conds:                       
#                     condhkl = cond.subs({h:hkl[0], k:hkl[1], l:hkl[2]}) # evaluation for hkl
#                     #print(hkl, cond, condhkl) ##########

#                     if not condhkl:   # skip if any condition violated
#                         break        # break out of conds loop as soon as one fails                                                
                                                       
#                 if condhkl: #condition is True
#                     if sp.N(rule.subs({h:hkl[0], k:hkl[1], l:hkl[2]}, simultaneous=True)) % 2 != 0: # expression not even; rule broken
#                         truthList[i+1] = False
#                         break   # break out of laue loop as soon as a rule fails
#                 #print(list(map(int, truthList))) ############
                
#         truthDict[tuple(hkl)] = truthList
    
#     return truthDict

def combos(N):
    # return list of all combinations of rules with rule 1 present in all cases
    from itertools import combinations

    _ll, _lc = [], [[1]]
    for n in range(1, N):
        _ll += list(combinations(range(2, N+1), n))

    for _comb in _ll:
        _lc += [[1] + list(_comb)]
        
    return _lc


# def isLaueEquivalent(hkl1, hkl2, laueSymList):
#     isEquivalent = False
#     for laueSym in laueSymList:
#         [h, k, l] = hkl1
#         if hkl2 == eval('[' + laueSym[0] + ',' + laueSym[1] + ',' + laueSym[2] + ']'):
#             isEquivalent = True
#             break
#     return isEquivalent




#### delete    
# def isAllowedByCondRules(hklList, rulesDict):
#     laueSymList, transRule, nonTransCondRules =   rulesDict['laueSymList'], rulesDict['transRule'], rulesDict['nonTransCondRules'] 
    
#     hklAllowed = []

#     #start with transrule ... seems to work but slow ... maybe put laue in outser loop?
    
#     for hkl in hklList:
#         #print(hkl)
#         allowed = True # assume true unless a rule is violated
#         # centring rule
#         for laue in laueSymList:

#             dummy, transRule =  apply_laueSym_to_cond_rule([{}, rulesDict['transRule']], laue)
                       
#             if sp.N(transRule.subs({h:hkl[0], k:hkl[1], l:hkl[2]})) % 2 != 0: # expression not even; rule broken
#                 allowed = False
#                 #print('False because trans rule failed: ', transRule, 'laue transrule:', transRule.subs({h:hkl[0], k:hkl[1], l:hkl[2]}))
#                 break
                
#             for nonTransCondRule in nonTransCondRules:
#                 conds, rule =  apply_laueSym_to_cond_rule(nonTransCondRule, laue)
                                                        
#                 for cond in conds:
#                     condhkl = cond.subs({h:hkl[0], k:hkl[1], l:hkl[2]}) # evaluation for hkl
#                     if not condhkl:   # skip if any condition violated
#                         break                                                        
                                                       
#                 if condhkl: #condition is True
#                     if sp.N(rule.subs({h:hkl[0], k:hkl[1], l:hkl[2]}, simultaneous=True)) % 2 != 0: # expression not even; rule broken
#                         allowed = False
#                         #print('False because non-trans rule failed. laue:', laue, 'original conds:', conds, 'original rule: ', rule )

#                         break          
                                                        
            
#         print(hkl, allowed)
#         hklAllowed += [allowed]
    
#     return hklAllowed

In [5]:
# #### tmp

# dd = dict()
# dd[tuple([1,2,3])] = [True, True, False, False]
# dd
# print(len(rulesDict['nonTransCondRules']))
# rulesDict['nonTransCondRules']
# truthListLength = 1 + len(rulesDict['nonTransCondRules'])
# truthList = [True] * truthListLength
# truthList[2] = False
# truthList
####

In [6]:
print(sg.space_group_info(sgNum).symbol_and_number())

symxyz = condRulesFromSpacegroup(sgNum)
print('symxyz:', symxyz)
condRulesAll = condRulesFromSymOps(symxyz)
#display(condRulesAll)
laueSymList = laueSymListFromSpacegroup(sgNum)
display('laueSymList:', laueSymList)
transRule, nonTransCondRules = separateTransRule(condRulesAll)
display('Trans rule:', transRule, 'Non Trans cond rules:', nonTransCondRules)
#for condRule in nonTransCondRules:
#    display(condRule, simplify_rule(condRule[1]))
    
rulesDict = {'laueSymList': laueSymList, 'transRule': transRule, 'nonTransCondRules': nonTransCondRules}    
########## delete these extra rules #######
#try ad-hoc rule l=0, expressed as l/100 = even
#nonTransCondRules += [[{sp.sympify(True)}, l/100]] ### rule no good

#nonTransCondRules += [[{sp.sympify(True)}, l/2]]
#display('>>>', nonTransCondRules) ###########

#nonTransCondRules += [[{sp.sympify(True)}, l/2 + 1]]
#display('>>>', nonTransCondRules) ########### 

#from Tables Wykoff a-f
nonTransCondRules += [[{sp.sympify(True)}, l - 1]]
nonTransCondRules += [[{sp.sympify(True)}, h + l/2]]
nonTransCondRules += [[{sp.sympify(True)}, h]]
nonTransCondRules += [[{sp.sympify(True)}, k]]
nonTransCondRules += [[{sp.sympify(True)}, h/2 + k/2 +l/2]]

###########################################

# these functions modify in place - careful!!!

print('-'*30)
display(nonTransCondRules)
display(remove_trivial_rules(simplify_rules(nonTransCondRules)))
display(remove_duplicate_rules(remove_trivial_rules(simplify_rules(nonTransCondRules))))

cond_rules = remove_duplicate_rules(remove_trivial_rules(simplify_rules(nonTransCondRules)))
display(cond_rules)

#print('-'*30)

#laue = laueSymList[9]
#cond_rule = cond_rules[8]
#new_cond_rule = apply_laueSym_to_cond_rule(cond_rule, laue)
#apply_laueSym_to_cond_rule()
#display(laue, cond_rule, new_cond_rule)


#print(sg.space_group_info(sgNum).symbol_and_number())
#print('Trans rule:', transRule)
#display('Other rules rule:', remove_laue_duplicates(cond_rules, laueSymList))

I 41/a c d :2 (No. 142)
symxyz: ['x,y,z', '-y+1/4,x+3/4,z+1/4', 'y+1/4,-x+1/4,z+3/4', 'x,-y,-z+1/2', '-x+1/2,y,-z', '-x,-y+1/2,z', 'y+1/4,x+3/4,-z+3/4', '-y+1/4,-x+1/4,-z+1/4', '-x,-y,-z', 'y-1/4,-x-3/4,-z-1/4', '-y-1/4,x-1/4,-z-3/4', '-x,y,z-1/2', 'x-1/2,-y,z', 'x,y-1/2,-z', '-y-1/4,-x-3/4,z-3/4', 'y-1/4,x-1/4,z-1/4', 'x+1/2,y+1/2,z+1/2', '-y+3/4,x+5/4,z+3/4', 'y+3/4,-x+3/4,z+5/4', 'x+1/2,-y+1/2,-z+1', '-x+1,y+1/2,-z+1/2', '-x+1/2,-y+1,z+1/2', 'y+3/4,x+5/4,-z+5/4', '-y+3/4,-x+3/4,-z+3/4', '-x+1/2,-y+1/2,-z+1/2', 'y+1/4,-x-1/4,-z+1/4', '-y+1/4,x+1/4,-z-1/4', '-x+1/2,y+1/2,z', 'x,-y+1/2,z+1/2', 'x+1/2,y,-z+1/2', '-y+1/4,-x-1/4,z-1/4', 'y+1/4,x+1/4,z+1/4']


'laueSymList:'

[['h', 'k', 'l'],
 ['-k', 'h', 'l'],
 ['-h', '-k', 'l'],
 ['k', '-h', 'l'],
 ['h', '-k', '-l'],
 ['k', 'h', '-l'],
 ['-h', 'k', '-l'],
 ['-k', '-h', '-l'],
 ['-h', '-k', '-l'],
 ['k', '-h', '-l'],
 ['h', 'k', '-l'],
 ['-k', 'h', '-l'],
 ['-h', 'k', 'l'],
 ['-k', '-h', 'l'],
 ['h', '-k', 'l'],
 ['k', 'h', 'l']]

'Trans rule:'

h + k + l

...

------------------------------


[[{Eq(h, 0), Eq(k, 0)}, l/2],
 [{Eq(h, 0), Eq(k, 0)}, 3*l/2],
 [{Eq(h - k, 0), Eq(l, 0)}, 2*k],
 [{Eq(h, 0)}, -l],
 [{Eq(k, 0)}, -h],
 [{Eq(l, 0)}, -k],
 [{Eq(h + k, 0)}, -k - 3*l/2],
 [{Eq(h - k, 0)}, -k - l/2],
 [{Eq(h, 0), Eq(k, 0)}, 3*l/2],
 [{Eq(h, 0), Eq(k, 0)}, 5*l/2],
 [{Eq(k, 0), Eq(l, 0)}, h],
 [{Eq(h, 0), Eq(l, 0)}, k],
 [{Eq(h, 0), Eq(k, 0)}, l],
 [{Eq(h - k, 0), Eq(l, 0)}, 4*k],
 [{Eq(h, 0)}, k],
 [{Eq(k, 0)}, l],
 [{Eq(l, 0)}, h],
 [{Eq(h + k, 0)}, -k - l/2],
 [{Eq(h - k, 0)}, k + l/2],
 [{True}, l - 1],
 [{True}, h + l/2],
 [{True}, h],
 [{True}, k],
 [{True}, h/2 + k/2 + l/2]]

[[{Eq(h, 0), Eq(k, 0)}, l/2],
 [{Eq(h, 0), Eq(k, 0)}, l/2],
 [{Eq(h, 0)}, l],
 [{Eq(k, 0)}, h],
 [{Eq(l, 0)}, k],
 [{Eq(h + k, 0)}, -k - 3*l/2],
 [{Eq(h - k, 0)}, -k - l/2],
 [{Eq(h, 0), Eq(k, 0)}, l/2],
 [{Eq(h, 0), Eq(k, 0)}, l/2],
 [{Eq(k, 0), Eq(l, 0)}, h],
 [{Eq(h, 0), Eq(l, 0)}, k],
 [{Eq(h, 0), Eq(k, 0)}, l],
 [{Eq(h, 0)}, k],
 [{Eq(k, 0)}, l],
 [{Eq(l, 0)}, h],
 [{Eq(h + k, 0)}, -k - l/2],
 [{Eq(h - k, 0)}, k + l/2],
 [{True}, l - 1],
 [{True}, h + l/2],
 [{True}, h],
 [{True}, k],
 [{True}, h/2 + k/2 + l/2]]

[[{Eq(h, 0), Eq(k, 0)}, l/2],
 [{Eq(h, 0)}, l],
 [{Eq(k, 0)}, h],
 [{Eq(l, 0)}, k],
 [{Eq(h + k, 0)}, -k - 3*l/2],
 [{Eq(h - k, 0)}, -k - l/2],
 [{Eq(k, 0), Eq(l, 0)}, h],
 [{Eq(h, 0), Eq(l, 0)}, k],
 [{Eq(h, 0), Eq(k, 0)}, l],
 [{Eq(h, 0)}, k],
 [{Eq(k, 0)}, l],
 [{Eq(l, 0)}, h],
 [{Eq(h + k, 0)}, -k - l/2],
 [{Eq(h - k, 0)}, k + l/2],
 [{True}, l - 1],
 [{True}, h + l/2],
 [{True}, h],
 [{True}, k],
 [{True}, h/2 + k/2 + l/2]]

[[{Eq(h, 0), Eq(k, 0)}, l/2],
 [{Eq(h, 0)}, l],
 [{Eq(k, 0)}, h],
 [{Eq(l, 0)}, k],
 [{Eq(h + k, 0)}, -k - 3*l/2],
 [{Eq(h - k, 0)}, -k - l/2],
 [{Eq(k, 0), Eq(l, 0)}, h],
 [{Eq(h, 0), Eq(l, 0)}, k],
 [{Eq(h, 0), Eq(k, 0)}, l],
 [{Eq(h, 0)}, k],
 [{Eq(k, 0)}, l],
 [{Eq(l, 0)}, h],
 [{Eq(h + k, 0)}, -k - l/2],
 [{Eq(h - k, 0)}, k + l/2],
 [{True}, l - 1],
 [{True}, h + l/2],
 [{True}, h],
 [{True}, k],
 [{True}, h/2 + k/2 + l/2]]

In [7]:
## get rules for group
## get reflist for tensor class
## .... calculate rules


import pickle

filename = '775af26c-38bd-4138-8e4b-867a1dcda47a' # full model hkl to 5
with open(filename,"rb") as f0:
    (reflist, out) = pickle.load(f0)



In [8]:
# for each sg/site, create a disctionary of classes of tensors    
for _sg in out:
    if _sg[1] != {}:
        #print('#%i' % sg[0])
        for site in _sg[1].keys():
            #print(site)
            #print(_sg[1][site])
            
            _sg[1][site]['classes'] = {}
            
            for ref in _sg[1][site]['ref'].keys():
                #print(ref)
                ranks = _sg[1][site]['ref'][ref]['ranks']
                #print(ranks)
    
                # is ref in existing class or need a new class?
                new_class = True
                for cls in _sg[1][site]['classes']:
                    #if ranks in _sg[1][site]['classes'][cls]['ranks']:
                    if ranks == _sg[1][site]['classes'][cls]['ranks']:
                        _sg[1][site]['classes'][cls]['refs'] += [ref]
                        new_class  = False
                    #else:
                    #    break
                
                if new_class:
                    if not bool(_sg[1][site]['classes']): # empty dict
                        class_no = 1
                    else:
                        class_no = max(_sg[1][site]['classes'].keys()) + 1 
                        
                    _sg[1][site]['classes'][class_no] = {'refs': [ref], 'ranks': ranks}
                    
                        


In [9]:
# get truth list for one spacegroup and save (time-consuming - load from pickle for test)


#filename = "tt_tmp_01aug" # this version contains the two ad-hoc rules
#filename = "tt_tmp_no_adhoc_2" # this version does not contain the two ad-hoc rules (l/2 = even and l/2+2 = even)
#filename = "tt_tmp_no_adhoc_3" # one ad-hoc rule: l/100 = even (i.e. l = 0)
#filename = "tt_tmp_no_adhoc_4" # single ad-hoc rule: l/2 = even
filename = "tt_tmp_adhoc_5" # five ad-hoc rules from Int Tables from Wykoff a-f
import pickle

# comment out to save new data
tt = getTruthListfromReflist(reflist, rulesDict)
with open(filename,"wb") as f0:
    pickle.dump(tt, f0)

# uncomment to use saved rules
#with open(filename,"rb") as f0:
#    tt = pickle.load(f0)

In [10]:
### get rules after removing duplicates from complete set; start with small sets and increase till one is not inconsistent

from sympy.logic import SOPform, POSform
from sympy import symbols

#all refs for #161 site a class 1
#allowed_list = out[sgNum-1][1]['a']['classes'][1]['refs'] #ok
#allowed_list = out[sgNum-1][1]['a']['classes'][2]['refs'] #ok but full rules give more complex (possibly euivalent) solution so small set is best
#allowed_list = out[sgNum-1][1]['d']['classes'][1]['refs'] #ok
#allowed_list = out[sgNum-1][1]['e']['classes'][5]['refs'] # no consistent tables for c1, c2, e1, e5 - reducing rule set didn't help

# look at c1 + c2 (don't work individually) # OK:  𝑟1∧(¬𝑟4∨¬𝑟6∨¬𝑟7)
#allowed_list = out[sgNum-1][1]['c']['classes'][1]['refs'] + out[sgNum-1][1]['c']['classes'][2]['refs']

# look at e1 + e5 (don't work individually) # OK: 𝑟1∧𝑟2∧¬𝑟7
#allowed_list = out[sgNum-1][1]['e']['classes'][1]['refs'] + out[sgNum-1][1]['e']['classes'][5]['refs']

allowed_list = out[sgNum-1][1]['c']['classes'][1]['refs']



#reflist

def allowed(hkl, allowed_list):
    return tuple(hkl) in allowed_list

allRules = [rulesDict['transRule']] + rulesDict['nonTransCondRules']

import numpy as np

truthListList = []
for ref in tt.keys():
    #print(ref, list(map(int,tt[ref])))
    truthListList += [list(map(int,tt[ref]))]
truthListListByRuleBig = np.array(truthListList).T.tolist() # includes duplicates



#remove duplictaes rules/truths
truthListListByRuleSmall, allRulesSmall = [], []
#truthListListByRuleSmall = []

for ruleNum in range(len(truthListListByRuleBig)):
    if truthListListByRuleBig[ruleNum] in truthListListByRuleSmall:
        print('Rule %i duplicated' % ruleNum)
    else:
        truthListListByRuleSmall += [truthListListByRuleBig[ruleNum]]
        allRulesSmall += [allRules[ruleNum]]

print('Rules used:')
for i in range(1, len(allRulesSmall)):
    print('r%i:\t' % i, allRulesSmall[i])
        
len(truthListListByRuleSmall)
### down to 9 rules...

# mofify getMinterms funtion
def getMinterms1(hkl_list, ruleTruths):
    
    N = len(ruleTruths)
    minterms, dontcares = [], []
    
    for i in range(2 ** N):
        truth = [bool(int(s)) for s in list(format(i, '0%sb' % N))]
        rules_allowed = None
        for ii in range(len(hkl_list)): ########## use index and not hkl
            if truth == [truthList[ii] for truthList in ruleTruths]:  #truth pattern matches pattern in truth table
                #if allowed(hkl_list[ii]) == rules_allowed or rules_allowed == None:
                if allowed(hkl_list[ii], allowed_list) == rules_allowed or rules_allowed == None:
                    #rules_allowed = allowed(hkl_list[ii])
                    rules_allowed = allowed(hkl_list[ii], allowed_list)
                    
                else:
                    raise Exception('inconsistent rules:', truth, ' same set of truths have reflections that are allowed and forbidden so no combination of the rules can work') # 
    
        if rules_allowed == None:
            dontcares += [truth]
        elif rules_allowed == True:
            minterms += [truth]
      
    return (minterms, dontcares)

N = len(truthListListByRuleSmall)
for rules_combination in combos(N):
    rules_comb_0 = [i-1 for i in rules_combination] # subtract 1 so rule 1 is list item 0
    smallrules = [truthListListByRuleSmall[i] for i in rules_comb_0]
    #getMinterms1(reflist, smallrules)
    #print(rules_combination)
    try:
        (minterms, dontcares) = getMinterms1(reflist, smallrules)
        symList = symbols(['r%i' % i for i in rules_combination]) # create a list of symbols, one per rule
        display(POSform(symList, minterms, dontcares))
        print('Found consistent combination of rules', rules_combination)
        print('====================', len(smallrules), len(minterms), len(dontcares))
        break # stop after the first consistent compound rule (remove break to compute all)
    except:
        pass
        #print('rules inconsistent')




# from sympy.logic import SOPform
# from sympy import symbols
# N = len(truthListListByRuleSmall)
# symList = symbols(['r%i' % i for i in range(1, N+1)]) # create a list of symbols, one per rule

# #(minterms, dontcares) =  getMinterms1(hkl_list, truthListListByRuleSmall)
# (minterms, dontcares) =  getMinterms1(reflist, truthListListByRuleSmall)
# #SOPform([r1, r2, r3], minterms, dontcares)
# print(minterms)
# print(dontcares)
# display(SOPform(symList, minterms, dontcares))
# display(POSform(symList, minterms, dontcares))

Rule 2 duplicated
Rule 8 duplicated
Rule 9 duplicated
Rule 10 duplicated
Rule 12 duplicated
Rule 14 duplicated
Rule 15 duplicated
Rule 16 duplicated
Rule 17 duplicated
Rule 18 duplicated
Rule 19 duplicated
Rule 23 duplicated
Rules used:
r1:	 [{Eq(h, 0), Eq(k, 0)}, l/2]
r2:	 [{Eq(h - k, 0), Eq(l, 0)}, 0]
r3:	 [{Eq(h, 0)}, l]
r4:	 [{Eq(k, 0)}, h]
r5:	 [{Eq(l, 0)}, k]
r6:	 [{Eq(h + k, 0)}, -k - 3*l/2]
r7:	 [{Eq(k, 0), Eq(l, 0)}, h]
r8:	 [{Eq(h, 0), Eq(k, 0)}, l]
r9:	 [{True}, l - 1]
r10:	 [{True}, h + l/2]
r11:	 [{True}, h]
r12:	 [{True}, h/2 + k/2 + l/2]


r1 & ~r12 & (~r4 | ~r6 | ~r7)

Found consistent combination of rules [1, 4, 6, 7, 12]
==================== 5 4 18


In [11]:
#### tmp to delete
display(allRules)
display(allRulesSmall)

[h + k + l,
 [{Eq(h, 0), Eq(k, 0)}, l/2],
 [{Eq(h, 0), Eq(k, 0)}, l/2],
 [{Eq(h - k, 0), Eq(l, 0)}, 0],
 [{Eq(h, 0)}, l],
 [{Eq(k, 0)}, h],
 [{Eq(l, 0)}, k],
 [{Eq(h + k, 0)}, -k - 3*l/2],
 [{Eq(h - k, 0)}, -k - l/2],
 [{Eq(h, 0), Eq(k, 0)}, l/2],
 [{Eq(h, 0), Eq(k, 0)}, l/2],
 [{Eq(k, 0), Eq(l, 0)}, h],
 [{Eq(h, 0), Eq(l, 0)}, k],
 [{Eq(h, 0), Eq(k, 0)}, l],
 [{Eq(h - k, 0), Eq(l, 0)}, 0],
 [{Eq(h, 0)}, k],
 [{Eq(k, 0)}, l],
 [{Eq(l, 0)}, h],
 [{Eq(h + k, 0)}, -k - l/2],
 [{Eq(h - k, 0)}, k + l/2],
 [{True}, l - 1],
 [{True}, h + l/2],
 [{True}, h],
 [{True}, k],
 [{True}, h/2 + k/2 + l/2]]

[h + k + l,
 [{Eq(h, 0), Eq(k, 0)}, l/2],
 [{Eq(h - k, 0), Eq(l, 0)}, 0],
 [{Eq(h, 0)}, l],
 [{Eq(k, 0)}, h],
 [{Eq(l, 0)}, k],
 [{Eq(h + k, 0)}, -k - 3*l/2],
 [{Eq(k, 0), Eq(l, 0)}, h],
 [{Eq(h, 0), Eq(k, 0)}, l],
 [{True}, l - 1],
 [{True}, h + l/2],
 [{True}, h],
 [{True}, h/2 + k/2 + l/2]]

In [12]:
### function form of previous cell - experimental - delete if doesn't work
###############################################################################



#all refs for #161 site a class 1
#allowed_list = out[sgNum-1][1]['a']['classes'][1]['refs'] #ok
#allowed_list = out[sgNum-1][1]['a']['classes'][2]['refs'] #ok but full rules give more complex (possibly euivalent) solution so small set is best
#allowed_list = out[sgNum-1][1]['d']['classes'][1]['refs'] #ok
#allowed_list = out[sgNum-1][1]['e']['classes'][5]['refs'] # no consistent tables for c1, c2, e1, e5 - reducing rule set didn't help

# look at c1 + c2 (don't work individually) # OK:  𝑟1∧(¬𝑟4∨¬𝑟6∨¬𝑟7)
#allowed_list = out[sgNum-1][1]['c']['classes'][1]['refs'] + out[sgNum-1][1]['c']['classes'][2]['refs']

# look at e1 + e5 (don't work individually) # OK: 𝑟1∧𝑟2∧¬𝑟7
#allowed_list = out[sgNum-1][1]['e']['classes'][1]['refs'] + out[sgNum-1][1]['e']['classes'][5]['refs']

#allowed_list = out[sgNum-1][1]['c']['classes'][1]['refs']



#reflist

def findCompoundRule(allowed_list):
    #pass more valiables (tt, reflist)

    from sympy.logic import SOPform, POSform
    from sympy import symbols

    def allowed(hkl, allowed_list):
        return tuple(hkl) in allowed_list

    allRules = [rulesDict['transRule']] + rulesDict['nonTransCondRules']

    import numpy as np

    truthListList = []
    for ref in tt.keys():
        #print(ref, list(map(int,tt[ref])))
        truthListList += [list(map(int,tt[ref]))]
    truthListListByRuleBig = np.array(truthListList).T.tolist() # includes duplicates



    #remove duplictaes rules/truths
    truthListListByRuleSmall, allRulesSmall = [], []
    #truthListListByRuleSmall = []

    for ruleNum in range(len(truthListListByRuleBig)):
        if truthListListByRuleBig[ruleNum] in truthListListByRuleSmall:
            print('Rule %i duplicated' % ruleNum)
        else:
            truthListListByRuleSmall += [truthListListByRuleBig[ruleNum]]
            allRulesSmall += [allRules[ruleNum]]

    print('Rules used:')
#    for i in range(1, len(allRulesSmall)):
#        print('r%i:\t' % i, allRulesSmall[i])
    for i in range(len(allRulesSmall)):
        print('r%i:\t' % (i + 1), allRulesSmall[i]) # modified to fix problem due to indexing from 0 while labelleing from 1

    len(truthListListByRuleSmall)
    ### down to 9 rules...

    # mofify getMinterms funtion
    def getMinterms1(hkl_list, ruleTruths):

        N = len(ruleTruths)
        minterms, dontcares = [], []

        for i in range(2 ** N):
            truth = [bool(int(s)) for s in list(format(i, '0%sb' % N))]
            rules_allowed = None
            for ii in range(len(hkl_list)): ########## use index and not hkl
                if truth == [truthList[ii] for truthList in ruleTruths]:  #truth pattern matches pattern in truth table
                    #if allowed(hkl_list[ii]) == rules_allowed or rules_allowed == None:
                    if allowed(hkl_list[ii], allowed_list) == rules_allowed or rules_allowed == None:
                        #rules_allowed = allowed(hkl_list[ii])
                        rules_allowed = allowed(hkl_list[ii], allowed_list)

                    else:
                        raise Exception('inconsistent rules:', truth, ' same set of truths have reflections that are allowed and forbidden so no combination of the rules can work') # 

            if rules_allowed == None:
                dontcares += [truth]
            elif rules_allowed == True:
                minterms += [truth]

        return (minterms, dontcares)

    N = len(truthListListByRuleSmall)
    for rules_combination in combos(N):
        rules_comb_0 = [i-1 for i in rules_combination] # subtract 1 so rule 1 is list item 0
        smallrules = [truthListListByRuleSmall[i] for i in rules_comb_0]
        #getMinterms1(reflist, smallrules)
        #print(rules_combination)
        try:
            (minterms, dontcares) = getMinterms1(reflist, smallrules)
            symList = symbols(['r%i' % i for i in rules_combination]) # create a list of symbols, one per rule
            compRule = POSform(symList, minterms, dontcares)
            display(compRule)
            #display(POSform(symList, minterms, dontcares))
            print('Found consistent combination of rules', rules_combination)
            print('====================', len(smallrules), len(minterms), len(dontcares))
            break # stop after the first consistent compound rule (remove break to compute all)
        except:
            compRule = None   #if can't find a compound rule
            #print('rules inconsistent')
    
    return compRule

    
for site in out[sgNum-1][1].keys():
    for class_no in out[sgNum-1][1][site]['classes'].keys():      
        print('\n\nSpacegroup #%i Site: %s Class: %i ranks: ' % (sgNum, site, class_no), out[sgNum-1][1][site]['classes'][class_no]['ranks'])
        findCompoundRule(out[sgNum-1][1][site]['classes'][class_no]['refs'])


        
# evaluate allRulesSmall for each ref in class

getTruthListfromReflist(out[sgNum-1][1][site]['classes'][class_no]['refs'], rulesDict)

        



Spacegroup #142 Site: a Class: 1 ranks:  [-2, -3, -4, -5, -6]
Rule 2 duplicated
Rule 8 duplicated
Rule 9 duplicated
Rule 10 duplicated
Rule 12 duplicated
Rule 14 duplicated
Rule 15 duplicated
Rule 16 duplicated
Rule 17 duplicated
Rule 18 duplicated
Rule 19 duplicated
Rule 23 duplicated
Rules used:
r1:	 h + k + l
r2:	 [{Eq(h, 0), Eq(k, 0)}, l/2]
r3:	 [{Eq(h - k, 0), Eq(l, 0)}, 0]
r4:	 [{Eq(h, 0)}, l]
r5:	 [{Eq(k, 0)}, h]
r6:	 [{Eq(l, 0)}, k]
r7:	 [{Eq(h + k, 0)}, -k - 3*l/2]
r8:	 [{Eq(k, 0), Eq(l, 0)}, h]
r9:	 [{Eq(h, 0), Eq(k, 0)}, l]
r10:	 [{True}, l - 1]
r11:	 [{True}, h + l/2]
r12:	 [{True}, h]
r13:	 [{True}, h/2 + k/2 + l/2]


r1 & (~r6 | ~r7)

Found consistent combination of rules [1, 6, 7]
==================== 3 3 1


Spacegroup #142 Site: a Class: 2 ranks:  [1, 3, 4, 5, 6, -2, -3, -4, -5, -6]
Rule 2 duplicated
Rule 8 duplicated
Rule 9 duplicated
Rule 10 duplicated
Rule 12 duplicated
Rule 14 duplicated
Rule 15 duplicated
Rule 16 duplicated
Rule 17 duplicated
Rule 18 duplicated
Rule 19 duplicated
Rule 23 duplicated
Rules used:
r1:	 h + k + l
r2:	 [{Eq(h, 0), Eq(k, 0)}, l/2]
r3:	 [{Eq(h - k, 0), Eq(l, 0)}, 0]
r4:	 [{Eq(h, 0)}, l]
r5:	 [{Eq(k, 0)}, h]
r6:	 [{Eq(l, 0)}, k]
r7:	 [{Eq(h + k, 0)}, -k - 3*l/2]
r8:	 [{Eq(k, 0), Eq(l, 0)}, h]
r9:	 [{Eq(h, 0), Eq(k, 0)}, l]
r10:	 [{True}, l - 1]
r11:	 [{True}, h + l/2]
r12:	 [{True}, h]
r13:	 [{True}, h/2 + k/2 + l/2]


r1 & ~r4

Found consistent combination of rules [1, 4]
==================== 2 1 0


Spacegroup #142 Site: b Class: 1 ranks:  [-2, -3, -4, -5, -6]
Rule 2 duplicated
Rule 8 duplicated
Rule 9 duplicated
Rule 10 duplicated
Rule 12 duplicated
Rule 14 duplicated
Rule 15 duplicated
Rule 16 duplicated
Rule 17 duplicated
Rule 18 duplicated
Rule 19 duplicated
Rule 23 duplicated
Rules used:
r1:	 h + k + l
r2:	 [{Eq(h, 0), Eq(k, 0)}, l/2]
r3:	 [{Eq(h - k, 0), Eq(l, 0)}, 0]
r4:	 [{Eq(h, 0)}, l]
r5:	 [{Eq(k, 0)}, h]
r6:	 [{Eq(l, 0)}, k]
r7:	 [{Eq(h + k, 0)}, -k - 3*l/2]
r8:	 [{Eq(k, 0), Eq(l, 0)}, h]
r9:	 [{Eq(h, 0), Eq(k, 0)}, l]
r10:	 [{True}, l - 1]
r11:	 [{True}, h + l/2]
r12:	 [{True}, h]
r13:	 [{True}, h/2 + k/2 + l/2]


...

r1 & r2 & (r6 | r7) & (~r4 | ~r6 | ~r7)

Found consistent combination of rules [1, 2, 4, 6, 7]
==================== 5 3 21


Spacegroup #142 Site: g Class: 3 ranks:  [2, 3, 4, 5, 6, -2, -3, -4, -5, -6]
Rule 2 duplicated
Rule 8 duplicated
Rule 9 duplicated
Rule 10 duplicated
Rule 12 duplicated
Rule 14 duplicated
Rule 15 duplicated
Rule 16 duplicated
Rule 17 duplicated
Rule 18 duplicated
Rule 19 duplicated
Rule 23 duplicated
Rules used:
r1:	 h + k + l
r2:	 [{Eq(h, 0), Eq(k, 0)}, l/2]
r3:	 [{Eq(h - k, 0), Eq(l, 0)}, 0]
r4:	 [{Eq(h, 0)}, l]
r5:	 [{Eq(k, 0)}, h]
r6:	 [{Eq(l, 0)}, k]
r7:	 [{Eq(h + k, 0)}, -k - 3*l/2]
r8:	 [{Eq(k, 0), Eq(l, 0)}, h]
r9:	 [{Eq(h, 0), Eq(k, 0)}, l]
r10:	 [{True}, l - 1]
r11:	 [{True}, h + l/2]
r12:	 [{True}, h]
r13:	 [{True}, h/2 + k/2 + l/2]


r1 & ~r2

Found consistent combination of rules [1, 2]
==================== 2 1 0


{(0, 0, -2): [True,
  False,
  False,
  True,
  True,
  True,
  True,
  False,
  False,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  False,
  False,
  True,
  True,
  False],
 (0, 0, 2): [True,
  False,
  False,
  True,
  True,
  True,
  True,
  False,
  False,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  False,
  False,
  True,
  True,
  False]}

In [13]:
#delete
cls = 2
out[sgNum-1][1]['c']['classes'][cls]
   
laueSymList


[['h', 'k', 'l'],
 ['-k', 'h', 'l'],
 ['-h', '-k', 'l'],
 ['k', '-h', 'l'],
 ['h', '-k', '-l'],
 ['k', 'h', '-l'],
 ['-h', 'k', '-l'],
 ['-k', '-h', '-l'],
 ['-h', '-k', '-l'],
 ['k', '-h', '-l'],
 ['h', 'k', '-l'],
 ['-k', 'h', '-l'],
 ['-h', 'k', 'l'],
 ['-k', '-h', 'l'],
 ['h', '-k', 'l'],
 ['k', 'h', 'l']]

In [14]:
allRulesSmall

[h + k + l,
 [{Eq(h, 0), Eq(k, 0)}, l/2],
 [{Eq(h - k, 0), Eq(l, 0)}, 0],
 [{Eq(h, 0)}, l],
 [{Eq(k, 0)}, h],
 [{Eq(l, 0)}, k],
 [{Eq(h + k, 0)}, -k - 3*l/2],
 [{Eq(k, 0), Eq(l, 0)}, h],
 [{Eq(h, 0), Eq(k, 0)}, l],
 [{True}, l - 1],
 [{True}, h + l/2],
 [{True}, h],
 [{True}, h/2 + k/2 + l/2]]

In [15]:
##################

In [16]:
#delete
#allowed_list_c1 = out[sgNum-1][1]['c']['classes'][1]['refs'] # no consistent tables for c1, c2, e1, e5 - reducing rule set didn't help
#allowed_list_c2 = out[sgNum-1][1]['c']['classes'][2]['refs'] # no consistent tables for c1, c2, e1, e5 - reducing rule set didn't help
#set(allowed_list_c1) & set(allowed_list_c2) # empty -ok
#allowed_list = allowed_list_c1 + allowed_list_c2

In [17]:
####################################

In [18]:
#allowed_list = out[sgNum-1][1]['c']['classes'][1]['refs'] # no consistent tables for c1, c2, e1, e5 - reducing rule set didn't help
#allowed_list = out[sgNum-1][1]['c']['classes'][2]['refs'] # no consistent tables for c1, c2, e1, e5 - reducing rule set didn't help
#allowed_list = out[sgNum-1][1]['e']['classes'][1]['refs'] # no consistent tables for c1, c2, e1, e5 - reducing rule set didn't help
allowed_list = out[sgNum-1][1]['e']['classes'][5]['refs'] # no consistent tables for c1, c2, e1, e5 - reducing rule set didn't help




#should be no refs that are allowed but volate R1

# num_refs = 0
# print('hkl\t\tR1\tallowed')
# for ii in range(len(reflist)):
#     if allowed(reflist[ii], allowed_list): #### print only allowed
#     #if True:                                #### print all refs
#         print(reflist[ii], '\t', smallrules[0][ii],'\t', int(allowed(reflist[ii], allowed_list)))
#         num_refs += 1
#     if int(allowed(reflist[ii], allowed_list)) and not smallrules[0][ii]:
#         raise('something went wrong with rule R1')        
# print(num_refs, ' reflections')




num_refs = 0
print('hkl\t\tallowed\tValid rules')
for ii in range(len(reflist)):
    if allowed(reflist[ii], allowed_list): #### print only allowed
    #if True:                                #### print all refs
        tlist = []
        for jj in range(len(smallrules)):
            tlist += [smallrules[jj][ii]]
        #print(reflist[ii], '\t', int(allowed(reflist[ii], allowed_list)), '\t', smallrules[0][ii], )
        print(reflist[ii], '\t', int(allowed(reflist[ii], allowed_list)), '\t', tlist)
        num_refs += 1
    if int(allowed(reflist[ii], allowed_list)) and not smallrules[0][ii]:
        raise('something went wrong with rule R1')        
print(num_refs, ' reflections')


######## fix this: keep only Laue unique reflections
###############
##############


num_refs = 0
print('hkl\t\tallowed\tValid rules')
refs = []
for ii in range(len(reflist)):
    #check to see if any of the Laue equivalents of reflist[ii] is already in refs
    laue_equiv = laueClassEquivRefs(reflist[ii], laueSymList)
    if set(tuple(i) for i in refs) & set(tuple(i) for i in laue_equiv) == set():
        #laue_equiv.sort()
        laue_equiv.sort(key = lambda hh : hh[0] + 0.99 * hh[1] + 0.98 * hh[2], reverse = True)
        ref = laue_equiv[0]
        refs += [ref]
        if allowed(reflist[ii], allowed_list): #### print only allowed
            tlist = []
            for jj in range(len(smallrules)):
                tlist += [smallrules[jj][ii]]
            #print(reflist[ii], '\t', int(allowed(reflist[ii], allowed_list)), '\t', smallrules[0][ii], )
            print(ref, '\t', int(allowed(reflist[ii], allowed_list)), '\t', tlist)
            num_refs += 1
print(num_refs, ' reflections')






#### all seem to satisfy R1
#### next:
#### show truth of all rules (R2, R3, R4...) - should be no simple patern (all true or all false or some function of rules that is all true)
####     not quite right: simple pattern might not give a rule as there may be reflections that are forbidden but satisfty the rule.
#### ignore reflections that are related thought Laue symmetry
#### find new rules by eye
#### compare to set of rules - what are the new ones?

hkl		allowed	Valid rules
[-2 -2 -2] 	 1 	 [1, 1, 1, 0, 1]
[-2 -2  2] 	 1 	 [1, 1, 1, 0, 1]
[-2  2 -2] 	 1 	 [1, 1, 1, 0, 1]
[-2  2  2] 	 1 	 [1, 1, 1, 0, 1]
[ 2 -2 -2] 	 1 	 [1, 1, 1, 0, 1]
[ 2 -2  2] 	 1 	 [1, 1, 1, 0, 1]
[ 2  2 -2] 	 1 	 [1, 1, 1, 0, 1]
[2 2 2] 	 1 	 [1, 1, 1, 0, 1]
[-4 -4 -2] 	 1 	 [1, 1, 1, 0, 1]
[-4 -4  2] 	 1 	 [1, 1, 1, 0, 1]
[-4  4 -2] 	 1 	 [1, 1, 1, 0, 1]
[-4  4  2] 	 1 	 [1, 1, 1, 0, 1]
[ 4 -4 -2] 	 1 	 [1, 1, 1, 0, 1]
[ 4 -4  2] 	 1 	 [1, 1, 1, 0, 1]
[ 4  4 -2] 	 1 	 [1, 1, 1, 0, 1]
[4 4 2] 	 1 	 [1, 1, 1, 0, 1]
16  reflections
hkl		allowed	Valid rules
[2, 2, 2] 	 1 	 [1, 1, 1, 0, 1]
[4, 4, 2] 	 1 	 [1, 1, 1, 0, 1]
2  reflections


In [19]:
laueSymList

[['h', 'k', 'l'],
 ['-k', 'h', 'l'],
 ['-h', '-k', 'l'],
 ['k', '-h', 'l'],
 ['h', '-k', '-l'],
 ['k', 'h', '-l'],
 ['-h', 'k', '-l'],
 ['-k', '-h', '-l'],
 ['-h', '-k', '-l'],
 ['k', '-h', '-l'],
 ['h', 'k', '-l'],
 ['-k', 'h', '-l'],
 ['-h', 'k', 'l'],
 ['-k', '-h', 'l'],
 ['h', '-k', 'l'],
 ['k', 'h', 'l']]

In [20]:
####################################

In [21]:
#display(rulesDict)
display(rulesDict['transRule'])
display(rulesDict['nonTransCondRules'][3])

h + k + l

[{Eq(h, 0)}, l]

In [22]:
### test rules derived above - temp code - delete later

#example d1
#allowed_list = out[sgNum-1][1]['d']['classes'][1]['refs']
allowed_list = out[sgNum-1][1]['a']['classes'][2]['refs']

def allowed(hkl, allowed_list):
    return tuple(hkl) in allowed_list


import numpy as np

truthListList = []
for ref in tt.keys():
    #print(ref, list(map(int,tt[ref])))
    truthListList += [list(map(int,tt[ref]))]
truthListListByRuleBig = np.array(truthListList).T.tolist() # includes duplicates

#remove duplictaes rules/truths
truthListListByRuleSmall = []
for ruleNum in range(len(truthListListByRuleBig)):
    if truthListListByRuleBig[ruleNum] in truthListListByRuleSmall:
        print('Rule %i duplicated' % ruleNum)
    else:
        truthListListByRuleSmall += [truthListListByRuleBig[ruleNum]]

len(truthListListByRuleSmall)

#example: d1
#compound_rule = np.logical_and(truthListListByRuleSmall[1 - 1], np.logical_not(truthListListByRuleSmall[6 - 1])) # R1^¬R6
compound_rule = np.logical_and(np.logical_not(truthListListByRuleSmall[4 - 1]), np.logical_not(truthListListByRuleSmall[5 - 1])) # R1^¬R6


n_errors = 0
for ii in range(len(reflist)):
    #print(reflist[ii], compound_rule[ii], allowed(reflist[ii], allowed_list))
    if compound_rule[ii] != allowed(reflist[ii], allowed_list):
        print(reflist[ii], compound_rule[ii], allowed(reflist[ii], allowed_list))
        n_errors += 1
print('No. of errors: ', n_errors)

#print('no. minterms: ', len(minterms))
#print('no. dontcares: ',len(dontcares))

##########################################
# 
# seems to work now (using np.logical_and): test other cases
# 
##########################################

Rule 2 duplicated
Rule 8 duplicated
Rule 9 duplicated
Rule 10 duplicated
Rule 12 duplicated
Rule 14 duplicated
Rule 15 duplicated
Rule 16 duplicated
Rule 17 duplicated
Rule 18 duplicated
Rule 19 duplicated
Rule 23 duplicated
No. of errors:  0


In [23]:
rulesDict

{'laueSymList': [['h', 'k', 'l'],
  ['-k', 'h', 'l'],
  ['-h', '-k', 'l'],
  ['k', '-h', 'l'],
  ['h', '-k', '-l'],
  ['k', 'h', '-l'],
  ['-h', 'k', '-l'],
  ['-k', '-h', '-l'],
  ['-h', '-k', '-l'],
  ['k', '-h', '-l'],
  ['h', 'k', '-l'],
  ['-k', 'h', '-l'],
  ['-h', 'k', 'l'],
  ['-k', '-h', 'l'],
  ['h', '-k', 'l'],
  ['k', 'h', 'l']],
 'transRule': h + k + l,
 'nonTransCondRules': [[{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h - k, 0), Eq(l, 0)}, 0],
  [{Eq(h, 0)}, l],
  [{Eq(k, 0)}, h],
  [{Eq(l, 0)}, k],
  [{Eq(h + k, 0)}, -k - 3*l/2],
  [{Eq(h - k, 0)}, -k - l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(k, 0), Eq(l, 0)}, h],
  [{Eq(h, 0), Eq(l, 0)}, k],
  [{Eq(h, 0), Eq(k, 0)}, l],
  [{Eq(h - k, 0), Eq(l, 0)}, 0],
  [{Eq(h, 0)}, k],
  [{Eq(k, 0)}, l],
  [{Eq(l, 0)}, h],
  [{Eq(h + k, 0)}, -k - l/2],
  [{Eq(h - k, 0)}, k + l/2],
  [{True}, l - 1],
  [{True}, h + l/2],
  [{True}, h],
  [{True}, k],
  [{True}, h/2 + k/2 + l/2]]

In [24]:
#rulesDict['nonTransCondRules'] += [[{}, l/2]]

In [25]:
rulesDict

{'laueSymList': [['h', 'k', 'l'],
  ['-k', 'h', 'l'],
  ['-h', '-k', 'l'],
  ['k', '-h', 'l'],
  ['h', '-k', '-l'],
  ['k', 'h', '-l'],
  ['-h', 'k', '-l'],
  ['-k', '-h', '-l'],
  ['-h', '-k', '-l'],
  ['k', '-h', '-l'],
  ['h', 'k', '-l'],
  ['-k', 'h', '-l'],
  ['-h', 'k', 'l'],
  ['-k', '-h', 'l'],
  ['h', '-k', 'l'],
  ['k', 'h', 'l']],
 'transRule': h + k + l,
 'nonTransCondRules': [[{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h - k, 0), Eq(l, 0)}, 0],
  [{Eq(h, 0)}, l],
  [{Eq(k, 0)}, h],
  [{Eq(l, 0)}, k],
  [{Eq(h + k, 0)}, -k - 3*l/2],
  [{Eq(h - k, 0)}, -k - l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(k, 0), Eq(l, 0)}, h],
  [{Eq(h, 0), Eq(l, 0)}, k],
  [{Eq(h, 0), Eq(k, 0)}, l],
  [{Eq(h - k, 0), Eq(l, 0)}, 0],
  [{Eq(h, 0)}, k],
  [{Eq(k, 0)}, l],
  [{Eq(l, 0)}, h],
  [{Eq(h + k, 0)}, -k - l/2],
  [{Eq(h - k, 0)}, k + l/2],
  [{True}, l - 1],
  [{True}, h + l/2],
  [{True}, h],
  [{True}, k],
  [{True}, h/2 + k/2 + l/2]]

In [26]:
### delete


#print(len(tt[(0,0,0)]))
#print(len(rulesDict['nonTransCondRules']))
#print(rulesDict['transRule'])
#print(rulesDict['nonTransCondRules'][3])

#first 20 refs for selected classes
print('c1')
print(out[sgNum-1][1]['c']['classes'][1]['refs'][0:20])
print('c2')
print(out[sgNum-1][1]['c']['classes'][2]['refs'][0:20])
print('e1')
print(out[sgNum-1][1]['e']['classes'][1]['refs'][0:20])
print('e5')
print(out[sgNum-1][1]['e']['classes'][5]['refs'][0:20])


def showTruths(hkl):
    truths = getTruthListfromReflist([hkl], rulesDict)
    print('hkl = ', hkl)
    print('\n', rulesDict['transRule'], ':\t', truths[hkl][0])
    print()
    for i in range(len(rulesDict['nonTransCondRules'])):
        assert len(rulesDict['nonTransCondRules']) == len(truths[hkl]) - 1
        print(rulesDict['nonTransCondRules'][i], ':\t', truths[hkl][i+1])
        print()

showTruths((1, 0, 1))
print('\n\n')
showTruths((0, 0, 2))




c1
[(-1, -1, 0), (-1, 0, -1), (-1, 0, 1), (-1, 1, 0), (0, -1, -1), (0, -1, 1), (0, 1, -1), (0, 1, 1), (1, -1, 0), (1, 0, -1), (1, 0, 1), (1, 1, 0), (-3, -1, 0), (-3, 0, -1), (-3, 0, 1), (-3, 1, 0), (-1, -3, 0), (-1, 0, -3), (-1, 0, 3), (-1, 3, 0)]
c2
[(0, 0, -2), (0, 0, 2), (-2, -2, -2), (-2, -2, 2), (-2, 2, -2), (-2, 2, 2), (2, -2, -2), (2, -2, 2), (2, 2, -2), (2, 2, 2), (-4, -4, -2), (-4, -4, 2), (-4, 4, -2), (-4, 4, 2), (4, -4, -2), (4, -4, 2), (4, 4, -2), (4, 4, 2)]
e1
[(-1, -1, 0), (-1, 1, 0), (1, -1, 0), (1, 1, 0), (-3, -3, 0), (-3, 3, 0), (-1, -1, -4), (-1, -1, 4), (-1, 1, -4), (-1, 1, 4), (1, -1, -4), (1, -1, 4), (1, 1, -4), (1, 1, 4), (3, -3, 0), (3, 3, 0), (-3, -3, -4), (-3, -3, 4), (-3, 3, -4), (-3, 3, 4)]
e5
[(-2, -2, -2), (-2, -2, 2), (-2, 2, -2), (-2, 2, 2), (2, -2, -2), (2, -2, 2), (2, 2, -2), (2, 2, 2), (-4, -4, -2), (-4, -4, 2), (-4, 4, -2), (-4, 4, 2), (4, -4, -2), (4, -4, 2), (4, 4, -2), (4, 4, 2)]
hkl =  (1, 0, 1)

 h + k + l :	 True

[{Eq(h, 0), Eq(k, 0)}, l/2] :	 

In [27]:
########## delete
display(allRulesSmall)
print(len(allRulesSmall))

print('Rules used:')
#for i in range(1, len(allRulesSmall)):
for i in range(0, len(allRulesSmall)):
    print('r%i:\t' % i, allRulesSmall[i])

########## rule 0 lost; lindex rules from 1?

print(combos(4))
print(rules_comb_0)

print('g1')
print(out[sgNum-1][1]['c']['classes'][1]['refs'])


[h + k + l,
 [{Eq(h, 0), Eq(k, 0)}, l/2],
 [{Eq(h - k, 0), Eq(l, 0)}, 0],
 [{Eq(h, 0)}, l],
 [{Eq(k, 0)}, h],
 [{Eq(l, 0)}, k],
 [{Eq(h + k, 0)}, -k - 3*l/2],
 [{Eq(k, 0), Eq(l, 0)}, h],
 [{Eq(h, 0), Eq(k, 0)}, l],
 [{True}, l - 1],
 [{True}, h + l/2],
 [{True}, h],
 [{True}, h/2 + k/2 + l/2]]

13
Rules used:
r0:	 h + k + l
r1:	 [{Eq(h, 0), Eq(k, 0)}, l/2]
r2:	 [{Eq(h - k, 0), Eq(l, 0)}, 0]
r3:	 [{Eq(h, 0)}, l]
r4:	 [{Eq(k, 0)}, h]
r5:	 [{Eq(l, 0)}, k]
r6:	 [{Eq(h + k, 0)}, -k - 3*l/2]
r7:	 [{Eq(k, 0), Eq(l, 0)}, h]
r8:	 [{Eq(h, 0), Eq(k, 0)}, l]
r9:	 [{True}, l - 1]
r10:	 [{True}, h + l/2]
r11:	 [{True}, h]
r12:	 [{True}, h/2 + k/2 + l/2]
[[1], [1, 2], [1, 3], [1, 4], [1, 2, 3], [1, 2, 4], [1, 3, 4], [1, 2, 3, 4]]
[0, 3, 5, 6, 11]
g1
[(-1, -1, 0), (-1, 0, -1), (-1, 0, 1), (-1, 1, 0), (0, -1, -1), (0, -1, 1), (0, 1, -1), (0, 1, 1), (1, -1, 0), (1, 0, -1), (1, 0, 1), (1, 1, 0), (-3, -1, 0), (-3, 0, -1), (-3, 0, 1), (-3, 1, 0), (-1, -3, 0), (-1, 0, -3), (-1, 0, 3), (-1, 3, 0), (0, -3, -1), (0, -3, 1), (0, -1, -3), (0, -1, 3), (0, 1, -3), (0, 1, 3), (0, 3, -1), (0, 3, 1), (1, -3, 0), (1, 0, -3), (1, 0, 3), (1, 3, 0), (3, -1, 0), (3, 0, -1), (3, 0, 1), (3, 1, 0), (-3, -3, 0), (-3, 0, -3), (-3, 0, 3), (-3, 3, 0), (-1, -1, -4), (-1, -1, 4), (-1, 1, -4), (-1, 1, 4), (0, -3, -3), (0,

## Label rules from 1 but index from 0
## Rules combos all to include rule 1 if primative but not therwise (currently only coded with one primative).
# Next:
## check Spacegroup #142 Site: g Class: 1 - why does is not require R1?

getTruthListfromReflist(out[sgNum-1][1][site]['classes'][class_no]['refs'], rulesDict)
- create small rulesDict (rules not duplicated) and run this command to check refs against rules.

## check: K=+1

(1,1,0)- allowed K=+1

(0,0,2) - forbidden K=+1

Look at tensor calc - why is this?


In [28]:
# create rules dict with small set of rules for testing and test against reflections in class

# multiple copies for test - delete extra copies

site, cls = 'c', 1

import copy
smallRulesDict = copy.copy(rulesDict)
display(smallRulesDict)
smallRulesDict['nonTransCondRules'] = allRulesSmall[1:] #miss out the first which is a transrule
display(smallRulesDict)

hkl_first = out[sgNum-1][1][site]['classes'][cls]['refs'][0] # just to get sizes
_truths = getTruthListfromReflist([hkl_first], smallRulesDict)
print('(h, k, l)\t' + '\t'.join(['R%i' % (i + 1) for i in range(len(_truths[hkl_first]))]))
for hkl in out[sgNum-1][1][site]['classes'][cls]['refs']:
    _truths = getTruthListfromReflist([hkl], smallRulesDict)
    print('(%i, %i, %i)\t' % hkl + '\t'.join(['%i' % int(i) for i in _truths[hkl]]))


{'laueSymList': [['h', 'k', 'l'],
  ['-k', 'h', 'l'],
  ['-h', '-k', 'l'],
  ['k', '-h', 'l'],
  ['h', '-k', '-l'],
  ['k', 'h', '-l'],
  ['-h', 'k', '-l'],
  ['-k', '-h', '-l'],
  ['-h', '-k', '-l'],
  ['k', '-h', '-l'],
  ['h', 'k', '-l'],
  ['-k', 'h', '-l'],
  ['-h', 'k', 'l'],
  ['-k', '-h', 'l'],
  ['h', '-k', 'l'],
  ['k', 'h', 'l']],
 'transRule': h + k + l,
 'nonTransCondRules': [[{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h - k, 0), Eq(l, 0)}, 0],
  [{Eq(h, 0)}, l],
  [{Eq(k, 0)}, h],
  [{Eq(l, 0)}, k],
  [{Eq(h + k, 0)}, -k - 3*l/2],
  [{Eq(h - k, 0)}, -k - l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(k, 0), Eq(l, 0)}, h],
  [{Eq(h, 0), Eq(l, 0)}, k],
  [{Eq(h, 0), Eq(k, 0)}, l],
  [{Eq(h - k, 0), Eq(l, 0)}, 0],
  [{Eq(h, 0)}, k],
  [{Eq(k, 0)}, l],
  [{Eq(l, 0)}, h],
  [{Eq(h + k, 0)}, -k - l/2],
  [{Eq(h - k, 0)}, k + l/2],
  [{True}, l - 1],
  [{True}, h + l/2],
  [{True}, h],
  [{True}, k],
  [{True}, h/2 + k/2 + l/2]]

{'laueSymList': [['h', 'k', 'l'],
  ['-k', 'h', 'l'],
  ['-h', '-k', 'l'],
  ['k', '-h', 'l'],
  ['h', '-k', '-l'],
  ['k', 'h', '-l'],
  ['-h', 'k', '-l'],
  ['-k', '-h', '-l'],
  ['-h', '-k', '-l'],
  ['k', '-h', '-l'],
  ['h', 'k', '-l'],
  ['-k', 'h', '-l'],
  ['-h', 'k', 'l'],
  ['-k', '-h', 'l'],
  ['h', '-k', 'l'],
  ['k', 'h', 'l']],
 'transRule': h + k + l,
 'nonTransCondRules': [[{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h - k, 0), Eq(l, 0)}, 0],
  [{Eq(h, 0)}, l],
  [{Eq(k, 0)}, h],
  [{Eq(l, 0)}, k],
  [{Eq(h + k, 0)}, -k - 3*l/2],
  [{Eq(k, 0), Eq(l, 0)}, h],
  [{Eq(h, 0), Eq(k, 0)}, l],
  [{True}, l - 1],
  [{True}, h + l/2],
  [{True}, h],
  [{True}, h/2 + k/2 + l/2]]}

(h, k, l)	R1	R2	R3	R4	R5	R6	R7	R8	R9	R10	R11	R12	R13
(-1, -1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-1, 0, -1)	1	1	1	0	0	1	1	1	1	1	0	0	0
(-1, 0, 1)	1	1	1	0	0	1	1	1	1	1	0	0	0
(-1, 1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(0, -1, -1)	1	1	1	0	0	1	1	1	1	1	0	0	0
(0, -1, 1)	1	1	1	0	0	1	1	1	1	1	0	0	0
(0, 1, -1)	1	1	1	0	0	1	1	1	1	1	0	0	0
(0, 1, 1)	1	1	1	0	0	1	1	1	1	1	0	0	0
(1, -1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(1, 0, -1)	1	1	1	0	0	1	1	1	1	1	0	0	0
(1, 0, 1)	1	1	1	0	0	1	1	1	1	1	0	0	0
(1, 1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-3, -1, 0)	1	1	1	1	1	0	1	1	1	0	0	0	0
(-3, 0, -1)	1	1	1	0	0	1	1	1	1	1	0	0	0
(-3, 0, 1)	1	1	1	0	0	1	1	1	1	1	0	0	0
(-3, 1, 0)	1	1	1	1	1	0	1	1	1	0	0	0	0
(-1, -3, 0)	1	1	1	1	1	0	1	1	1	0	0	0	0
(-1, 0, -3)	1	1	1	0	0	1	1	1	1	1	0	0	0
(-1, 0, 3)	1	1	1	0	0	1	1	1	1	1	0	0	0
(-1, 3, 0)	1	1	1	1	1	0	1	1	1	0	0	0	0
(0, -3, -1)	1	1	1	0	0	1	1	1	1	1	0	0	0
(0, -3, 1)	1	1	1	0	0	1	1	1	1	1	0	0	0
(0, -1, -3)	1	1	1	0	0	1	1	1	1	1	0	0	0
(0, -1, 3)	1	1	1	0	0	1	1	1	1	1	0	0	0
(0, 1, -3)	1	1	1	0	0	1	1	1	1	1	0	0	0
(0, 1, 3)	1	1	1	

In [29]:
#here123
# delete cell

#display(smallRulesDict)
#display(getTruthListfromReflist([(0, 1, 1)], smallRulesDict))
display(getTruthListfromReflist([(2, 2, 2)], smallRulesDict))
#4, 6, 10, 11 are all wrong

# hkl = (0, 1, 1)
# rule = 4
# #get first item from cond set
# for cond in smallRulesDict['nonTransCondRules'][rule -2][0]:
#     break


# display(cond)
# display(cond.subs({h:hkl[0], k:hkl[1], l:hkl[2]}))

# rule = smallRulesDict['nonTransCondRules'][rule -2][1]
# display(rule)
# sp.N(rule.subs({h:hkl[0], k:hkl[1], l:hkl[2]}, simultaneous=True)) % 2 != 0 # expression not even; rule broken
# # correct - True because rule is broken
# #so what is wrong??????

# ######## check indexing and extra rules at end ???


{(2, 2, 2): [True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  False,
  False,
  True,
  False]}

In [30]:
# create rules dict with small set of rules for testing and test against reflections in class

site, cls = 'c', 2

import copy
smallRulesDict = copy.copy(rulesDict)
display(smallRulesDict)
smallRulesDict['nonTransCondRules'] = allRulesSmall[1:] #miss out the first which is a transrule
display(smallRulesDict)

hkl_first = out[sgNum-1][1][site]['classes'][cls]['refs'][0] # just to get sizes
_truths = getTruthListfromReflist([hkl_first], smallRulesDict)
print('(h, k, l)\t' + '\t'.join(['R%i' % (i + 1) for i in range(len(_truths[hkl_first]))]))
for hkl in out[sgNum-1][1][site]['classes'][cls]['refs']:
    _truths = getTruthListfromReflist([hkl], smallRulesDict)
    print('(%i, %i, %i)\t' % hkl + '\t'.join(['%i' % int(i) for i in _truths[hkl]]))

{'laueSymList': [['h', 'k', 'l'],
  ['-k', 'h', 'l'],
  ['-h', '-k', 'l'],
  ['k', '-h', 'l'],
  ['h', '-k', '-l'],
  ['k', 'h', '-l'],
  ['-h', 'k', '-l'],
  ['-k', '-h', '-l'],
  ['-h', '-k', '-l'],
  ['k', '-h', '-l'],
  ['h', 'k', '-l'],
  ['-k', 'h', '-l'],
  ['-h', 'k', 'l'],
  ['-k', '-h', 'l'],
  ['h', '-k', 'l'],
  ['k', 'h', 'l']],
 'transRule': h + k + l,
 'nonTransCondRules': [[{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h - k, 0), Eq(l, 0)}, 0],
  [{Eq(h, 0)}, l],
  [{Eq(k, 0)}, h],
  [{Eq(l, 0)}, k],
  [{Eq(h + k, 0)}, -k - 3*l/2],
  [{Eq(h - k, 0)}, -k - l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(k, 0), Eq(l, 0)}, h],
  [{Eq(h, 0), Eq(l, 0)}, k],
  [{Eq(h, 0), Eq(k, 0)}, l],
  [{Eq(h - k, 0), Eq(l, 0)}, 0],
  [{Eq(h, 0)}, k],
  [{Eq(k, 0)}, l],
  [{Eq(l, 0)}, h],
  [{Eq(h + k, 0)}, -k - l/2],
  [{Eq(h - k, 0)}, k + l/2],
  [{True}, l - 1],
  [{True}, h + l/2],
  [{True}, h],
  [{True}, k],
  [{True}, h/2 + k/2 + l/2]]

{'laueSymList': [['h', 'k', 'l'],
  ['-k', 'h', 'l'],
  ['-h', '-k', 'l'],
  ['k', '-h', 'l'],
  ['h', '-k', '-l'],
  ['k', 'h', '-l'],
  ['-h', 'k', '-l'],
  ['-k', '-h', '-l'],
  ['-h', '-k', '-l'],
  ['k', '-h', '-l'],
  ['h', 'k', '-l'],
  ['-k', 'h', '-l'],
  ['-h', 'k', 'l'],
  ['-k', '-h', 'l'],
  ['h', '-k', 'l'],
  ['k', 'h', 'l']],
 'transRule': h + k + l,
 'nonTransCondRules': [[{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h - k, 0), Eq(l, 0)}, 0],
  [{Eq(h, 0)}, l],
  [{Eq(k, 0)}, h],
  [{Eq(l, 0)}, k],
  [{Eq(h + k, 0)}, -k - 3*l/2],
  [{Eq(k, 0), Eq(l, 0)}, h],
  [{Eq(h, 0), Eq(k, 0)}, l],
  [{True}, l - 1],
  [{True}, h + l/2],
  [{True}, h],
  [{True}, h/2 + k/2 + l/2]]}

(h, k, l)	R1	R2	R3	R4	R5	R6	R7	R8	R9	R10	R11	R12	R13
(0, 0, -2)	1	0	1	1	1	1	0	1	1	0	0	1	0
(0, 0, 2)	1	0	1	1	1	1	0	1	1	0	0	1	0
(-2, -2, -2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(-2, -2, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(-2, 2, -2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(-2, 2, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(2, -2, -2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(2, -2, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(2, 2, -2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(2, 2, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(-4, -4, -2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(-4, -4, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(-4, 4, -2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(-4, 4, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(4, -4, -2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(4, -4, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(4, 4, -2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(4, 4, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0


In [31]:
# create rules dict with small set of rules for testing and test against reflections in class

site, cls = 'e', 1

import copy
smallRulesDict = copy.copy(rulesDict)
display(smallRulesDict)
smallRulesDict['nonTransCondRules'] = allRulesSmall[1:] #miss out the first which is a transrule
display(smallRulesDict)

hkl_first = out[sgNum-1][1][site]['classes'][cls]['refs'][0] # just to get sizes
_truths = getTruthListfromReflist([hkl_first], smallRulesDict)
print('(h, k, l)\t' + '\t'.join(['R%i' % (i + 1) for i in range(len(_truths[hkl_first]))]))
for hkl in out[sgNum-1][1][site]['classes'][cls]['refs']:
    _truths = getTruthListfromReflist([hkl], smallRulesDict)
    print('(%i, %i, %i)\t' % hkl + '\t'.join(['%i' % int(i) for i in _truths[hkl]]))

{'laueSymList': [['h', 'k', 'l'],
  ['-k', 'h', 'l'],
  ['-h', '-k', 'l'],
  ['k', '-h', 'l'],
  ['h', '-k', '-l'],
  ['k', 'h', '-l'],
  ['-h', 'k', '-l'],
  ['-k', '-h', '-l'],
  ['-h', '-k', '-l'],
  ['k', '-h', '-l'],
  ['h', 'k', '-l'],
  ['-k', 'h', '-l'],
  ['-h', 'k', 'l'],
  ['-k', '-h', 'l'],
  ['h', '-k', 'l'],
  ['k', 'h', 'l']],
 'transRule': h + k + l,
 'nonTransCondRules': [[{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h - k, 0), Eq(l, 0)}, 0],
  [{Eq(h, 0)}, l],
  [{Eq(k, 0)}, h],
  [{Eq(l, 0)}, k],
  [{Eq(h + k, 0)}, -k - 3*l/2],
  [{Eq(h - k, 0)}, -k - l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(k, 0), Eq(l, 0)}, h],
  [{Eq(h, 0), Eq(l, 0)}, k],
  [{Eq(h, 0), Eq(k, 0)}, l],
  [{Eq(h - k, 0), Eq(l, 0)}, 0],
  [{Eq(h, 0)}, k],
  [{Eq(k, 0)}, l],
  [{Eq(l, 0)}, h],
  [{Eq(h + k, 0)}, -k - l/2],
  [{Eq(h - k, 0)}, k + l/2],
  [{True}, l - 1],
  [{True}, h + l/2],
  [{True}, h],
  [{True}, k],
  [{True}, h/2 + k/2 + l/2]]

{'laueSymList': [['h', 'k', 'l'],
  ['-k', 'h', 'l'],
  ['-h', '-k', 'l'],
  ['k', '-h', 'l'],
  ['h', '-k', '-l'],
  ['k', 'h', '-l'],
  ['-h', 'k', '-l'],
  ['-k', '-h', '-l'],
  ['-h', '-k', '-l'],
  ['k', '-h', '-l'],
  ['h', 'k', '-l'],
  ['-k', 'h', '-l'],
  ['-h', 'k', 'l'],
  ['-k', '-h', 'l'],
  ['h', '-k', 'l'],
  ['k', 'h', 'l']],
 'transRule': h + k + l,
 'nonTransCondRules': [[{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h - k, 0), Eq(l, 0)}, 0],
  [{Eq(h, 0)}, l],
  [{Eq(k, 0)}, h],
  [{Eq(l, 0)}, k],
  [{Eq(h + k, 0)}, -k - 3*l/2],
  [{Eq(k, 0), Eq(l, 0)}, h],
  [{Eq(h, 0), Eq(k, 0)}, l],
  [{True}, l - 1],
  [{True}, h + l/2],
  [{True}, h],
  [{True}, h/2 + k/2 + l/2]]}

(h, k, l)	R1	R2	R3	R4	R5	R6	R7	R8	R9	R10	R11	R12	R13
(-1, -1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-1, 1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(1, -1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(1, 1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-3, -3, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-3, 3, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-1, -1, -4)	1	1	1	1	1	1	0	1	1	0	0	0	0
(-1, -1, 4)	1	1	1	1	1	1	0	1	1	0	0	0	0
(-1, 1, -4)	1	1	1	1	1	1	0	1	1	0	0	0	0
(-1, 1, 4)	1	1	1	1	1	1	0	1	1	0	0	0	0
(1, -1, -4)	1	1	1	1	1	1	0	1	1	0	0	0	0
(1, -1, 4)	1	1	1	1	1	1	0	1	1	0	0	0	0
(1, 1, -4)	1	1	1	1	1	1	0	1	1	0	0	0	0
(1, 1, 4)	1	1	1	1	1	1	0	1	1	0	0	0	0
(3, -3, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(3, 3, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-3, -3, -4)	1	1	1	1	1	1	0	1	1	0	0	0	0
(-3, -3, 4)	1	1	1	1	1	1	0	1	1	0	0	0	0
(-3, 3, -4)	1	1	1	1	1	1	0	1	1	0	0	0	0
(-3, 3, 4)	1	1	1	1	1	1	0	1	1	0	0	0	0
(3, -3, -4)	1	1	1	1	1	1	0	1	1	0	0	0	0
(3, -3, 4)	1	1	1	1	1	1	0	1	1	0	0	0	0
(3, 3, -4)	1	1	1	1	1	1	0	1	1	0	0	0	0
(3, 3, 4)	1	1	1	1	1	1	0	1	1	0	0	0	0
(-5, -5, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-5, 5, 0)	1	

In [32]:
# create rules dict with small set of rules for testing and test against reflections in class

site, cls = 'e', 5

import copy
smallRulesDict = copy.copy(rulesDict)
display(smallRulesDict)
smallRulesDict['nonTransCondRules'] = allRulesSmall[1:] #miss out the first which is a transrule
display(smallRulesDict)

hkl_first = out[sgNum-1][1][site]['classes'][cls]['refs'][0] # just to get sizes
_truths = getTruthListfromReflist([hkl_first], smallRulesDict)
print('(h, k, l)\t' + '\t'.join(['R%i' % (i + 1) for i in range(len(_truths[hkl_first]))]))
for hkl in out[sgNum-1][1][site]['classes'][cls]['refs']:
    _truths = getTruthListfromReflist([hkl], smallRulesDict)
    print('(%i, %i, %i)\t' % hkl + '\t'.join(['%i' % int(i) for i in _truths[hkl]]))

{'laueSymList': [['h', 'k', 'l'],
  ['-k', 'h', 'l'],
  ['-h', '-k', 'l'],
  ['k', '-h', 'l'],
  ['h', '-k', '-l'],
  ['k', 'h', '-l'],
  ['-h', 'k', '-l'],
  ['-k', '-h', '-l'],
  ['-h', '-k', '-l'],
  ['k', '-h', '-l'],
  ['h', 'k', '-l'],
  ['-k', 'h', '-l'],
  ['-h', 'k', 'l'],
  ['-k', '-h', 'l'],
  ['h', '-k', 'l'],
  ['k', 'h', 'l']],
 'transRule': h + k + l,
 'nonTransCondRules': [[{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h - k, 0), Eq(l, 0)}, 0],
  [{Eq(h, 0)}, l],
  [{Eq(k, 0)}, h],
  [{Eq(l, 0)}, k],
  [{Eq(h + k, 0)}, -k - 3*l/2],
  [{Eq(h - k, 0)}, -k - l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(k, 0), Eq(l, 0)}, h],
  [{Eq(h, 0), Eq(l, 0)}, k],
  [{Eq(h, 0), Eq(k, 0)}, l],
  [{Eq(h - k, 0), Eq(l, 0)}, 0],
  [{Eq(h, 0)}, k],
  [{Eq(k, 0)}, l],
  [{Eq(l, 0)}, h],
  [{Eq(h + k, 0)}, -k - l/2],
  [{Eq(h - k, 0)}, k + l/2],
  [{True}, l - 1],
  [{True}, h + l/2],
  [{True}, h],
  [{True}, k],
  [{True}, h/2 + k/2 + l/2]]

{'laueSymList': [['h', 'k', 'l'],
  ['-k', 'h', 'l'],
  ['-h', '-k', 'l'],
  ['k', '-h', 'l'],
  ['h', '-k', '-l'],
  ['k', 'h', '-l'],
  ['-h', 'k', '-l'],
  ['-k', '-h', '-l'],
  ['-h', '-k', '-l'],
  ['k', '-h', '-l'],
  ['h', 'k', '-l'],
  ['-k', 'h', '-l'],
  ['-h', 'k', 'l'],
  ['-k', '-h', 'l'],
  ['h', '-k', 'l'],
  ['k', 'h', 'l']],
 'transRule': h + k + l,
 'nonTransCondRules': [[{Eq(h, 0), Eq(k, 0)}, l/2],
  [{Eq(h - k, 0), Eq(l, 0)}, 0],
  [{Eq(h, 0)}, l],
  [{Eq(k, 0)}, h],
  [{Eq(l, 0)}, k],
  [{Eq(h + k, 0)}, -k - 3*l/2],
  [{Eq(k, 0), Eq(l, 0)}, h],
  [{Eq(h, 0), Eq(k, 0)}, l],
  [{True}, l - 1],
  [{True}, h + l/2],
  [{True}, h],
  [{True}, h/2 + k/2 + l/2]]}

(h, k, l)	R1	R2	R3	R4	R5	R6	R7	R8	R9	R10	R11	R12	R13
(-2, -2, -2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(-2, -2, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(-2, 2, -2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(-2, 2, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(2, -2, -2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(2, -2, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(2, 2, -2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(2, 2, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(-4, -4, -2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(-4, -4, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(-4, 4, -2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(-4, 4, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(4, -4, -2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(4, -4, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(4, 4, -2)	1	1	1	1	1	1	0	1	1	0	0	1	0
(4, 4, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0


In [33]:
hkl_first = out[sgNum-1][1][site]['classes'][cls]['refs'][0] # just to get sizes
_truths = getTruthListfromReflist([hkl_first], smallRulesDict)
print('(h, k, l)\t' + '\t'.join(['R%i' % (i + 1) for i in range(len(_truths[hkl_first]))]))
for hkl in [(1, 1, 0), (1, 1, 2),(2, 2, 2)]:
    _truths = getTruthListfromReflist([hkl], smallRulesDict)
    print('(%i, %i, %i)\t' % hkl + '\t'.join(['%i' % int(i) for i in _truths[hkl]]))

(h, k, l)	R1	R2	R3	R4	R5	R6	R7	R8	R9	R10	R11	R12	R13
(1, 1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(1, 1, 2)	1	1	1	1	1	1	1	1	1	0	1	0	0
(2, 2, 2)	1	1	1	1	1	1	0	1	1	0	0	1	0


In [34]:
hkl = (1, 1, 0); truths_110 = getTruthListfromReflist([hkl], smallRulesDict)[hkl]
print(truths_110)
truths_110[-1] = False
print(truths_110)

[True, True, True, True, True, False, False, True, True, False, False, False, False]
[True, True, True, True, True, False, False, True, True, False, False, False, False]


In [35]:
site_cls_list = [
    ['a', 1], ['a',2],
    ['b', 1], ['b',2], 
    ['c', 1], ['c',2],
    ['d', 1], ['d',2], ['d',3], 
    ['e', 1], ['e',2], ['e', 3], ['e',4], ['e', 5],
    ['f', 1], ['f',2], ['f',3], 
    ['g', 1], ['g',2], ['g',3],
]

for site_cls in site_cls_list:
    site, cls = site_cls

    for hkl in out[sgNum-1][1][site]['classes'][cls]['refs']:
        _truths = getTruthListfromReflist([hkl], smallRulesDict)
        if _truths[hkl] == truths_110:
            print('(%i, %i, %i)\t' % hkl + '\t'.join(['%i' % int(i) for i in _truths[hkl]]))

(-1, -1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-1, 1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(1, -1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(1, 1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-3, -3, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-3, 3, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(3, -3, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(3, 3, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-5, -5, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-5, 5, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(5, -5, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(5, 5, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-1, -1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-1, 1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(1, -1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(1, 1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-3, -3, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-3, 3, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(3, -3, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(3, 3, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-5, -5, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-5, 5, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(5, -5, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(5, 5, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-1, -1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(-1, 1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0
(1, -1, 0)	1	1	1	1	1	0	0	1	1	0	0	0	0


In [36]:
site_cls_list = [
    ['a', 1], ['a',2],
    ['b', 1], ['b',2], 
    ['c', 1], ['c',2],
    ['d', 1], ['d',2], ['d',3], 
    ['e', 1], ['e',2], ['e', 3], ['e',4], ['e', 5],
    ['f', 1], ['f',2], ['f',3], 
    ['g', 1], ['g',2], ['g',3],
]
site, cls = 'e', 5

In [37]:
site_cls_list

[['a', 1],
 ['a', 2],
 ['b', 1],
 ['b', 2],
 ['c', 1],
 ['c', 2],
 ['d', 1],
 ['d', 2],
 ['d', 3],
 ['e', 1],
 ['e', 2],
 ['e', 3],
 ['e', 4],
 ['e', 5],
 ['f', 1],
 ['f', 2],
 ['f', 3],
 ['g', 1],
 ['g', 2],
 ['g', 3]]